<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M0.532 · Pattern Recognition</p>
<p style="margin: 0; text-align:right;">Computational Engineering and Mathematics Master</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Computers, Multimedia and Telecommunications Department</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# Non linear filtering


In [ ]:
# import OpenCV library
import cv2

# we will use the following import to display images in colab:
from google.colab.patches import cv2_imshow

# we will use numpy to generate the kernel
import numpy as np


In [ ]:
!wget https://github.com/opencv/opencv/blob/master/samples/data/baboon.jpg?raw=true -O baboon.jpg

#read image
img_gray = cv2.imread('baboon.jpg', cv2.IMREAD_GRAYSCALE)
img = cv2.imread('baboon.jpg', cv2.IMREAD_COLOR)

## Median filter

Median filtering selects the median value from each pixel’s neighborhood. It is used to remove salt and pepper noise (noise that takes extreme white or black values in certain pixels in the image)

### Adding salt and pepper noise

We take a [function](https://stackoverflow.com/questions/22937589/how-to-add-noise-gaussian-salt-and-pepper-etc-to-image-in-python-with-opencv) to generate salt and pepper noise:

In [ ]:
def noisy(noise_typ,image):

  '''
  image : ndarray
      Input image data. Will be converted to float.
  mode : str
      One of the following strings, selecting the type of noise to add:

      'gauss'     Gaussian-distributed additive noise.
      'poisson'   Poisson-distributed noise generated from the data.
      's&p'       Replaces random pixels with 0 or 1.
      'speckle'   Multiplicative noise using out = image + n*image,where
                  n is uniform noise with specified mean & variance.
  '''

  if noise_typ == "gauss":
    row,col,ch= image.shape
    mean = 0
    var = 0.1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss
    return noisy

  elif noise_typ == "s&p":
    row,col,ch = image.shape
    s_vs_p = 0.5
    amount = 0.004
    out = np.copy(image)

    # Salt mode
    num_salt = int(np.ceil(amount * image.size * s_vs_p))

    # image salted
    for i in range(0, num_salt):
        row = np.random.randint(0, image.shape[0])
        col = np.random.randint(0, image.shape[1])
        out[row, col, :] = 255

    # Pepper mode
    num_pepper = int(np.ceil(amount* image.size * (1. - s_vs_p)))

    # image peppered
    for i in range(0,num_pepper):
        row = np.random.randint(0, image.shape[0])
        col = np.random.randint(0, image.shape[1])
        out[row, col, :] = 0

    return out

  elif noise_typ == "poisson":
    vals = len(np.unique(image))
    vals = 2 ** np.ceil(np.log2(vals))
    noisy = np.random.poisson(image * vals) / float(vals)
    return noisy

  elif noise_typ =="speckle":
    row,col,ch = image.shape
    gauss = np.random.randn(row,col,ch)
    gauss = gauss.reshape(row,col,ch)
    noisy = image + image * gauss
    return noisy

In [ ]:
img_with_noise = noisy("s&p", img)

In [ ]:
cv2_imshow(img)

In [ ]:
cv2_imshow(img_with_noise)

Since we are doing the examples with gray images, lets transform the image to gray with the [cvtColor](https://docs.opencv.org/4.5.4/d8/d01/group__imgproc__color__conversions.html#ga397ae87e1288a81d2363b61574eb8cab) function:

In [ ]:
 img_with_noise_gray = cv2.cvtColor(img_with_noise, cv2.COLOR_RGB2GRAY)

In [ ]:
cv2_imshow(img_with_noise_gray)

### Remove noise with median filter:

Use the [medianBlur](https://docs.opencv.org/4.5.4/d4/d86/group__imgproc__filter.html#ga564869aa33e58769b4469101aac458f9) function from Opencv. It has as a parameter the aperture size; it must be odd and greater than 1 (3, 5, 7 , ...)

In [ ]:

aperture_size = 5

median = cv2.medianBlur(img_with_noise_gray, aperture_size)


In [ ]:
cv2_imshow(median)

## Bilateral filter

Similarly at the median filtering, Owe can do the bilateral filtering with OpenCV. Bilateral filter is a filter to reduce noise while trying to preserve the edges sharp).


Looking at the documentation:

https://docs.opencv.org/4.5.4/d4/d86/group__imgproc__filter.html#ga9d7064d478c95d60003cf839430737ed


Python:

    cv.bilateralFilter(	src, d, sigmaColor, sigmaSpace[, dst[, borderType]]	) ->	dst


**d**	Diameter of each pixel neighborhood that is used during filtering.

**Sigma values**: For simplicity, you can set the 2 sigma values to be the same. If they are small (< 10), the filter will not have much effect, whereas if they are large (> 150), they will have a very strong effect, making the image look "cartoonish".


In [ ]:
d_bilateral = 7
sigma_bilateral = 70

img_bilateral = cv2.bilateralFilter(img_with_noise_gray, d_bilateral, sigma_bilateral, sigma_bilateral)

In [ ]:
cv2_imshow(img_bilateral)

Can you distinguish the results of the median filter from the bilateral one? Which one is better? Test with different parameters for each filter

## Connected components

Connected components are defined as regions of adjacent pixels that have the same input value or label. It works with binary images. To do so we first will binarize the image (using thresholding)


Lets load a suitable image for the analysis and [binarize](https://docs.opencv.org/4.5.4/d7/d1b/group__imgproc__misc.html#gae8a4a146d1ca78c626a53577199e9c57) it to do the example:

In [ ]:
!wget https://github.com/opencv/opencv/blob/master/samples/data/pic3.png?raw=true -O pic3.png

# read image
img = cv2.imread('pic3.png', cv2.IMREAD_GRAYSCALE)

# and convert to binary (pixels lower to 200 set to 0, and the others to 255):
img = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY)[1]

In [ ]:
cv2_imshow(img)

We can see that there are 4 shapes drawn in the image!

The [connected components](https://docs.opencv.org/4.5.4/d3/dc0/group__imgproc__shape.html#gaedef8c7340499ca391d459122e51bef5) function computes the connected components labeled image of boolean image

Python:
  >  cv.connectedComponents(	image[, labels[, connectivity[, ltype]]]	) ->	retval, labels

 returns
   > retval: the total number of labels [0, N-1] where 0 represents the background label.
   
   > labels: the resulting label of each pixel in the image

In [ ]:
num_labels, labels = cv2.connectedComponents(img)


In [ ]:
print(num_labels)

This result indicates that we have 6 connected labels (background + 5 regions)

In [ ]:

def imshow_connected_components(labels):

    # Map component labels to distinct colors (hsv):
    label_hue = np.uint8(179*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # cvt to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

    # set background label to black
    labeled_img[label_hue==0] = 0

    cv2_imshow(labeled_img)



In [ ]:
imshow_connected_components(labels)

See that the background areas separate the other regions!

### Distance transform

The distance transform is another binary image processing technique useful in quickly precomputing the distance to a curve. We can follow with the same image that we worked in the previous example:

In [ ]:
cv2_imshow(img)

The [distance transform](https://docs.opencv.org/4.5.4/d7/d1b/group__imgproc__misc.html#ga8a0b7fdfcb7a13dde018988ba3a43042) Calculates the distance to the closest zero pixel for each pixel of the source image:

Python:
> cv.distanceTransform(	src, distanceType, maskSize[, dst[, dstType]]	) ->	dst






In [ ]:
# Perform the distance transform algorithm

# distance_type: the simple euclidean distance
distance_type = cv2.DIST_L2
mask_size = 3
dist = cv2.distanceTransform(img, distance_type, mask_size)


In [ ]:
# Normalize the distance image for range = {0.0, 1.0}
# so we can visualize and threshold it
cv2.normalize(dist, dist, 0, 255, cv2.NORM_MINMAX)

In [ ]:
cv2_imshow(dist)

We can see that we have obtained the distance to background pixels. Whiter pixels are further from the background than the others